# Grupo 1 - Smog predicition
## Modelo SVM

### Análisis y limipieza de datos

In [ ]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

df = pd.read_csv('data/train.csv')

df['Gears'] = df['Transmission'].str.extract('(\d+)')
df['Gears'] = pd.to_numeric(df['Gears'], errors='coerce')
df['Transmission'] = df['Transmission'].str.extract('(\D+)')

In [ ]:
#Fuel Type
df.loc[df["Fuel Type"] == "X", "Fuel Type"] = 0
df.loc[df["Fuel Type"] == "Z", "Fuel Type"] = 1
df.loc[df["Fuel Type"] == "D", "Fuel Type"] = 2
df.loc[df["Fuel Type"] == "E", "Fuel Type"] = 3
df.loc[df["Fuel Type"] == "N", "Fuel Type"] = 4

#Transmission
df.loc[df["Transmission"] == "A", "Transmission"] = 0
df.loc[df["Transmission"] == "AM", "Transmission"] = 1
df.loc[df["Transmission"] == "AS", "Transmission"] = 2
df.loc[df["Transmission"] == "AV", "Transmission"] = 3
df.loc[df["Transmission"] == "M", "Transmission"] = 4


#Vehicle Class
df.loc[df["Vehicle Class"] == "Compact", "Vehicle Class"] = 0
df.loc[df["Vehicle Class"] == "Full-size", "Vehicle Class"] = 1
df.loc[df["Vehicle Class"] == "Mid-size", "Vehicle Class"] = 2
df.loc[df["Vehicle Class"] == "Minicompact", "Vehicle Class"] = 3
df.loc[df["Vehicle Class"] == "Minivan", "Vehicle Class"] = 4
df.loc[df["Vehicle Class"] == "Minicompact", "Vehicle Class"] = 5
df.loc[df["Vehicle Class"] == "Pickup truck: Small", "Vehicle Class"] = 6
df.loc[df["Vehicle Class"] == "Pickup truck: Standard", "Vehicle Class"] = 7
df.loc[df["Vehicle Class"] == "SUV: Small", "Vehicle Class"] = 8
df.loc[df["Vehicle Class"] == "SUV: Standard", "Vehicle Class"] = 9
df.loc[df["Vehicle Class"] == "Special purpose vehicle", "Vehicle Class"] = 10
df.loc[df["Vehicle Class"] == "Station wagon: Mid-size", "Vehicle Class"] = 11
df.loc[df["Vehicle Class"] == "Station wagon: Small", "Vehicle Class"] = 12
df.loc[df["Vehicle Class"] == "Subcompact", "Vehicle Class"] = 13
df.loc[df["Vehicle Class"] == "Two-seater", "Vehicle Class"] = 14

df['Gears'].fillna(df['Gears'].mean(), inplace=True)

In [ ]:
df.drop("Model Year", axis=1, inplace=True)
df.drop("Make", axis=1, inplace=True)
df.drop("Model", axis=1, inplace=True)
df.drop("Comb (mpg)", axis=1, inplace=True)
df.drop("Fuel Consumption City (L/100 km)", axis=1, inplace=True)
df.drop("Hwy (L/100 km)", axis=1, inplace=True)

df.head()

Con esto, hemos terminado el análisis y la limpieza de los datos de ambos dataframes.

### Entrenamiento del modelo

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)

%matplotlib inline

In [ ]:
features = ['Vehicle Class', 'Engine Size (L)', 'Cylinders', 'Transmission', 'Fuel Type', 'Comb (L/100 km)', 'CO2 Emissions (g/km)', 'Gears']

x = df[features].values
y = df['Smog'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=33)

In [ ]:
kernel = 'linear'
gamma = 3.0

model = SVC(kernel=kernel, probability=True, gamma=gamma)

Tras definir el modelo se va a pasar a la fase de entrenamiento.

In [ ]:
model.fit(x_train, y_train)

predicted = model.predict(x_test)
expected = y_test

### Comprobación de resultados

Para la comprobación de resultados se van a calcular varias métricas, obteniendo la eficacia de nuestro modelo.

In [ ]:
metrics.accuracy_score(expected, predicted)

Nuestra exactitud es bastante mala... Obteniendo un 52% de media.

In [ ]:
s_y_test = Series(y_test)
s_y_test.value_counts()

s_y_test.value_counts().head(1) / len(y_test)

Al cumplir la exactitud nula podemos deducir que el modelo no se encuentra sobreajustado a nuestro conjunto de datos.

In [ ]:
print(classification_report(expected, predicted))

El resto de métricas calculadas también son muy negativas...

Ahora, vamos a probar a entrenarlo y evaluarlo con K-Fold para ver si puede mejorar los resultados obtenidos.

In [ ]:

cv = KFold(n_splits=5, shuffle=True, random_state=33)

scores = cross_val_score(model, x, y, cv=cv)
print("Scores in every iteration", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Ha mejorado ligeramente pero sigue siendo un resultado muy negativo...